In [ ]:
_from_tweet = 1200000
_packet_size = 10

In [7]:
import nltk
nltk.download('stopwords')
!pip install pytypo
!pip install pyspellchecker
!pip install bs4

[nltk_data] Downloading package stopwords to /home/mgmdi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd  
import numpy as np
_cols = ['sentiment','id','date','query_string','user','text']
_df = pd.read_csv('../input/sentiment140/training.1600000.processed.noemoticon.csv',header=None, names=_cols, encoding = 'ISO-8859-1', low_memory=False)
_df = _df.drop(columns=['date', 'query_string','user'])
_df.head()

In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import WordNetLemmatizer 
from bs4 import BeautifulSoup
from spellchecker import SpellChecker
import string
import re
import pytypo
import json

# Tokenizers
_tweetok = TweetTokenizer()
_wptok = WordPunctTokenizer()

# Contractions Dictionary
_contraction_dict = json.load(open('../input/contraction-dict/contraction_dict.json', 'r'))

# Stop Words
_stop_words = set(stopwords.words('english'))
_exclude = ["not", "no", "should"]
_stop_words = [w for w in _stop_words if not w in _exclude]

# Regular Expressions
_r1 = r'@[A-Za-z0-9_]+' # Twitter username
_r2 = r'http\S+|www\S+|[A-Za-z0-9\/.:]*\.com(\.[A-Za-z]+)*' # URLs
_r3 = r'#[A-Za-z0-9]+' # HashTags
_final_regex = r'|'.join((_r1, _r2, _r3)) # Final RegEx

# Spell Checker
_spell = SpellChecker()
_spell.word_frequency.load_words(['microsoft', 'apple', 'google','twitter', 'facebook', 'amazon', 'texting', 'nah'])

# Lemmatization
_lemmatizer = WordNetLemmatizer()

IOError: [Errno 2] No such file or directory: '../input/contraction-dict/contraction_dict.json'

In [ ]:
def preprocess_tweet(valTweet):
    
    # HTML decoding
    valTweet = BeautifulSoup(valTweet, 'lxml').get_text()
    
    # Weird characters
    valTweet = ''.join(filter(lambda x: x in string.printable, valTweet))
    
    # URLs, mentions and hashtags
    valTweet = re.sub(_final_regex, '', valTweet)
    
    # To lowercase
    valTweet = valTweet.lower()
    
    # Expand contractions
    vTokens = _tweetok.tokenize(valTweet)
    valTweet = ' '.join([_contraction_dict[w] if w in _contraction_dict else w for w in vTokens]).strip()
    
    # Only alpha characters
    valTweet = re.sub(r'[^a-zA-Z]', ' ', valTweet)
    
    # Typos correction
    vTokens = _wptok.tokenize(valTweet)
    vTokens = [pytypo.correct(w) for w in vTokens]
    
    # Lonely letters removal
    vTokens = [w for w in vTokens if len(w) > 1]
    
    return vTokens

In [ ]:
def correct_spelling(valTokens):
    
    # Discard null tweet
    if(len(valTokens) == 0):
        return ""
    
    # Long word check, discard tweet if there is a long word
    if(len([w for w in valTokens if len(w) > 40]) > 0):
        return ""
    
    # Misspelled words correction
    vMisspelled = _spell.unknown(valTokens)
    if(len(vMisspelled) > 0):
        valTokens = [w if not w in vMisspelled else _spell.correction(w) for w in valTokens]
        # Stopword drop
        
    valTokens = [w for w in valTokens if not w in _stop_words]
        
    return valTokens

In [4]:
def lemmatize(valTokens):
    return [_lemmatizer.lemmatize(w) for w in valTokens] 

In [2]:
def remove_repeated(valTokens):
    return ' '.join(sorted(set(valTokens), key=valTokens.index))

In [ ]:
_texts = []
_sentiments = []
_ids = []
aux = _from_tweet + _packet_size
_to_tweet = aux if aux < len(_df['text']) else len(_df['text'])
for i in range(_from_tweet,_to_tweet):
    vTweet = preprocess_tweet(_df['text'][i])
    vTweet = correct_spelling(vTweet)
    vTweet = lemmatize(vTweet)
    vTweet = remove_repeated(vTweet)
    if vTweet != "":
        _texts.append(vTweet)
        _sentiments.append(_df['sentiment'][i])
        _ids.append(_df['id'][i])

In [ ]:
_cleaner_df = pd.DataFrame(_texts,columns=['text'])
_cleaner_df['sentiment'] = _sentiments
_cleaner_df['id'] = _ids
_cleaner_df.to_csv('cooked_tweets_' + str(_from_tweet) + '_' + str(_to_tweet) + '.csv', index = None, encoding='utf-8')